# This notebook will generate the Plausibility and Faithfulness results for the models

## The Expainability metrices are based on the work by DeYoung et. al. (2020) [ERASER: A Benchmark to Evaluate Rationalized NLP Models]

In [ ]:
# model choices include 'bert','bert_supervised','birnn','cnngru','birnn_att','birnn_scrat'
# attention lambda for bert_supervised is 0.001 and birnn_scrat is 100
# for testing_with_lime.py set number of samples is 100 (for faster inference set it lower but the results might not be consistent)



#python testing_with_rational.py <model> <att_lambda>
!python testing_with_rational.py bert_supervised 0.001
#python testing_with_lime.py <model> <number of samples> <att_lambda>
!python testing_with_lime.py bert_supervised 100 0.001

In [5]:
!python testing_with_rational.py cnngru 0.01

2023-08-18 13:24:48.746846: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2023-08-18 13:24:48.746887: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/root/.pyenv/versions/3.7.12/envs/.venv3.7/lib/python3.7/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))
Reading twitter - 1grams ...
Reading twitter - 2grams ...
/root/.pyenv/versions/3.7.12/envs/.venv3.7/lib/python3.7/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in
Reading english - 1grams ...
Using TensorFlow backend.
Since you dont want to use GPU, using the CPU instead.
1

## <font color='red'>NOTE: Please generate the model explainability output before running this notebook</font>

In [1]:
import json
from tqdm.notebook import tqdm
import more_itertools as mit
import os

In [2]:
# get_annotated_data method is used to load the dataset
from Preprocess import *
from Preprocess.dataCollect import *

/root/.pyenv/versions/3.7.12/envs/.venv3.7/lib/python3.7/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...


/root/.pyenv/versions/3.7.12/envs/.venv3.7/lib/python3.7/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading english - 1grams ...


2023-08-18 14:22:01.283281: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2023-08-18 14:22:01.283342: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
dict_data_folder={
      '2':{'data_file':'Data/dataset.json','class_label':'Data/classes_two.npy'},
      '3':{'data_file':'Data/dataset.json','class_label':'Data/classes.npy'}
}

# We need to load the dataset with the labels as 'hatespeech', 'offensive', and 'normal' (3-class). 

params = {}
params['num_classes']=2
params['data_file']=dict_data_folder[str(params['num_classes'])]['data_file']
params['class_names']=dict_data_folder[str(params['num_classes'])]['class_label']

data_all_labelled=get_annotated_data(params)

In [4]:
# The important key here is the 'bert_token'. Set it to True for Bert based models and False for Others.

params_data={
    'include_special':False,  #True is want to include <url> in place of urls if False will be removed
    'bert_tokens':True, #True /False
    'type_attention':'softmax', #softmax
    'set_decay':0.1,
    'majority':2,
    'max_length':128,
    'variance':10,
    'window':4,
    'alpha':0.5,
    'p_value':0.8,
    'method':'additive',
    'decay':False,
    'normalized':False,
    'not_recollect':True,
}


if(params_data['bert_tokens']):
    print('Loading BERT tokenizer...')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=False)
else:
    print('Loading Normal tokenizer...')
    tokenizer=None

Loading BERT tokenizer...


In [5]:
# Load the whole dataset and get the tokenwise rationales
def get_training_data(data):
    post_ids_list=[]
    text_list=[]
    attention_list=[]
    label_list=[]
    
    final_binny_output = []
    print('total_data',len(data))
    for index,row in tqdm(data.iterrows(),total=len(data)):
        annotation=row['final_label']
        
        text=row['text']
        post_id=row['post_id']
        annotation_list=[row['label1'],row['label2'],row['label3']]
        tokens_all = list(row['text'])
#         attention_masks =  [list(row['explain1']),list(row['explain2']),list(row['explain1'])]
        
        if(annotation!= 'undecided'):
            tokens_all,attention_masks=returnMask(row, params_data, tokenizer)
            final_binny_output.append([post_id, annotation, tokens_all, attention_masks, annotation_list])

    return final_binny_output



In [6]:
training_data=get_training_data(data_all_labelled)

  1%|▏         | 156/11874 [00:00<00:18, 623.84it/s]

total_data 11874


  5%|▌         | 636/11874 [00:00<00:12, 892.59it/s]

length of text ==0


 39%|███▉      | 4679/11874 [00:07<00:11, 603.08it/s]

...

 55%|█████▍    | 6486/11874 [00:09<00:09, 575.29it/s]

length of text ==0


 60%|██████    | 7140/11874 [00:10<00:07, 629.46it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors
100%|█████████▉| 11823/11874 [00:17<00:00, 662.60it/s]

length of text ==0


100%|██████████| 11874/11874 [00:17<00:00, 659.73it/s]


In [7]:
training_data[19]

['19',
 'non-toxic',
 [101, 100, 102],
 [[0, 0, 0], [0, 0, 0], [0, 0, 0]],
 ['normal', 'normal', 'normal']]

In [8]:
# https://stackoverflow.com/questions/2154249/identify-groups-of-continuous-numbers-in-a-list
def find_ranges(iterable):
    """Yield range of consecutive numbers."""
    for group in mit.consecutive_groups(iterable):
        group = list(group)
        if len(group) == 1:
            yield group[0]
        else:
            yield group[0], group[-1]
            
# Convert dataset into ERASER format: https://github.com/jayded/eraserbenchmark/blob/master/rationale_benchmark/utils.py
def get_evidence(post_id, anno_text, explanations):
    output = []

    indexes = sorted([i for i, each in enumerate(explanations) if each==1])
    span_list = list(find_ranges(indexes))

    for each in span_list:
        if type(each)== int:
            start = each
            end = each+1
        elif len(each) == 2:
            start = each[0]
            end = each[1]+1
        else:
            print('error')

        output.append({"docid":post_id, 
              "end_sentence": -1, 
              "end_token": end, 
              "start_sentence": -1, 
              "start_token": start, 
              "text": ' '.join([str(x) for x in anno_text[start:end]])})
    return output

# To use the metrices defined in ERASER, we will have to convert the dataset
def convert_to_eraser_format(dataset, method, save_split, save_path, id_division):  
    final_output = []
    
    if save_split:
        train_fp = open(save_path+'train.jsonl', 'w')
        val_fp = open(save_path+'val.jsonl', 'w')
        test_fp = open(save_path+'test.jsonl', 'w')
            
    for tcount, eachrow in enumerate(dataset):
        
        temp = {}
        post_id = eachrow[0]
        post_class = eachrow[1]
        anno_text_list = eachrow[2]
        majority_label = eachrow[1]
        
        if majority_label=='normal':
            continue
        
        all_labels = eachrow[4]
        explanations = []
        for each_explain in eachrow[3]:
            explanations.append(list(each_explain))
        
        # For this work, we have considered the union of explanations. Other options could be explored as well.
        if method == 'union':
            final_explanation = [any(each) for each in zip(*explanations)]
            final_explanation = [int(each) for each in final_explanation]
        
            
        temp['annotation_id'] = post_id
        temp['classification'] = post_class
        temp['evidences'] = [get_evidence(post_id, list(anno_text_list), final_explanation)]
        temp['query'] = "What is the class?"
        temp['query_type'] = None
        final_output.append(temp)
        
        if save_split:
            if not os.path.exists(save_path+'docs'):
                os.makedirs(save_path+'docs')
            
            with open(save_path+'docs/'+post_id, 'w') as fp:
                fp.write(' '.join([str(x) for x in list(anno_text_list)]))
            
            if post_id in id_division['train']:
                train_fp.write(json.dumps(temp)+'\n')
            
            elif post_id in id_division['val']:
                val_fp.write(json.dumps(temp)+'\n')
            
            elif post_id in id_division['test']:
                test_fp.write(json.dumps(temp)+'\n')
            else:
                print(post_id)
    
    if save_split:
        train_fp.close()
        val_fp.close()
        test_fp.close()
        
    return final_output

In [9]:
# The post_id_divisions file stores the train, val, test split ids. We select only the test ids.
with open('./Data/post_id_divisions.json') as fp:
    id_division = json.load(fp)

In [11]:
method = 'union'
save_split = True
save_path = './Data/Evaluation/Model_Eval/'  #The dataset in Eraser Format will be stored here.
convert_to_eraser_format(training_data, method, save_split, save_path, id_division)

[{'annotation_id': '0',
  'classification': 'non-toxic',
  'evidences': [[]],
  'query': 'What is the class?',
  'query_type': None},
 {'annotation_id': '1',
  'classification': 'non-toxic',
  'evidences': [[]],
  'query': 'What is the class?',
  'query_type': None},
 {'annotation_id': '2',
  'classification': 'non-toxic',
  'evidences': [[]],
  'query': 'What is the class?',
  'query_type': None},
 {'annotation_id': '3',
  'classification': 'non-toxic',
  'evidences': [[]],
  'query': 'What is the class?',
  'query_type': None},
 {'annotation_id': '4',
  'classification': 'non-toxic',
  'evidences': [[]],
  'query': 'What is the class?',
  'query_type': None},
 {'annotation_id': '5',
  'classification': 'non-toxic',
  'evidences': [[]],
  'query': 'What is the class?',
  'query_type': None},
 {'annotation_id': '6',
  'classification': 'non-toxic',
  'evidences': [[]],
  'query': 'What is the class?',
  'query_type': None},
 {'annotation_id': '7',
  'classification': 'non-toxic',
  'ev

### Generate the output

In [ ]:
%cd eraserbenchmark-master

In [ ]:
!ls

In [1]:
%cd eraserbenchmark

/mnt/c/Users/muham/Desktop/18-8-2023 thesis/HateXplain-master/HateXplain-master/eraserbenchmark


In [6]:
#--data_dir : Location of the folder which contains the dataset in eraser format
#--results : The location of the model output file in eraser format
#--score_file : The file name and location to write the output

!PYTHONPATH=./:$PYTHONPATH python rationale_benchmark/metrics.py --split test --strict --data_dir ../Data/Evaluation/Model_Eval --results ../explanations_dicts/bestModel_bert_base_uncased_Attn_train_TRUE_0.001_explanation_top5.json --score_file ../model_explain_output_bert_supervised.json

  3626 MainThread Error in instances: 0 instances fail validation: set()
/root/.pyenv/versions/3.7.12/envs/.venv3.7/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
  5945 MainThread No sentence level predictions detected, skipping sentence-level diagnostic
============label_to_int============
{'non-toxic': 0, 'toxic': 1}
============label_to_int============
{'48': {'annotation_id': '48', 'classification': 'normal', 'classification_scores': {'hatespeech': 0.4682544469833374, 'normal': 0.5317456126213074}, 'rationales': [{'docid': '48', 'hard_rationale_predictions': [{'end_token': 1, 'start_token': 0}, {'end_token': 9, 'start_token': 8}, {'end_token': 2, 'start_token': 1}, {'end_token': 3, 'start_token': 2}, {'end_token': 10, 'start_token': 9}], 'soft_rationale_predictions': [0.07330799847841263, 0.10276181250810623, 0.11001800745725632, 0.02011631615459919, 0.032124266028404236, 0.049152325838

In [23]:
#--data_dir : Location of the folder which contains the dataset in eraser format
#--results : The location of the model output file in eraser format
#--score_file : The file name and location to write the output

!PYTHONPATH=./:$PYTHONPATH python rationale_benchmark/metrics.py --split test --data_dir ../Data/Evaluation/Model_Eval --results ../explanations_dicts/bestModel_birnn_0.01_explanation_top5.json --score_file ../model_explain_output_birn.json

  7738 MainThread Error! For instance annotation=48, docid=48 expected classifications for 10 tokens but have them for 2 tokens instead!
  7738 MainThread Error! For instance annotation=76, docid=76 expected classifications for 7 tokens but have them for 2 tokens instead!
  7738 MainThread Error! For instance annotation=98, docid=98 expected classifications for 7 tokens but have them for 2 tokens instead!
  7739 MainThread Error! For instance annotation=102, docid=102 expected classifications for 8 tokens but have them for 2 tokens instead!
  7739 MainThread Error! For instance annotation=253, docid=253 expected classifications for 11 tokens but have them for 2 tokens instead!
  7739 MainThread Error! For instance annotation=284, docid=284 expected classifications for 21 tokens but have them for 2 tokens instead!
  7739 MainThread Error! For instance annotation=289, docid=289 expected classifications for 9 tokens but have them for 2 tokens instead!
  7739 MainThread Error! For instance

In [18]:
import json

In [19]:
# print the required results
with open('../model_explain_output_bert.json') as fp:
    output_data = json.load(fp)

print('\nPlausibility')
print('IOU F1 :', output_data['iou_scores'][0]['macro']['f1'])
print('Token F1 :', output_data['token_prf']['instance_macro']['f1'])
print('AUPRC :', output_data['token_soft_metrics']['auprc'])

print('\nFaithfulness')
print('Comprehensiveness :', output_data['classification_scores']['comprehensiveness'])
print('Sufficiency', output_data['classification_scores']['sufficiency'])


Plausibility
IOU F1 : 0.04949297609079916
Token F1 : 0.30057502003518816
AUPRC : nan

Faithfulness
Comprehensiveness : -0.0034962869753503495
Sufficiency -0.017371423684867323


In [ ]:
# print the required results
with open('../model_explain_output_cnn_gru.json') as fp:
    output_data = json.load(fp)

print('\nPlausibility')
print('IOU F1 :', output_data['iou_scores'][0]['macro']['f1'])
print('Token F1 :', output_data['token_prf']['instance_macro']['f1'])
print('AUPRC :', output_data['token_soft_metrics']['auprc'])

print('\nFaithfulness')
print('Comprehensiveness :', output_data['classification_scores']['comprehensiveness'])
print('Sufficiency', output_data['classification_scores']['sufficiency'])

In [7]:
import json
# print the required results
with open('../model_explain_output_bert_supervised.json') as fp:
    output_data = json.load(fp)

print('\nPlausibility')
print('IOU F1 :', output_data['iou_scores'][0]['macro']['f1'])
print('Token F1 :', output_data['token_prf']['instance_macro']['f1'])
print('AUPRC :', output_data['token_soft_metrics']['auprc'])

print('\nFaithfulness')
print('Comprehensiveness :', output_data['classification_scores']['comprehensiveness'])
print('Sufficiency', output_data['classification_scores']['sufficiency'])


Plausibility
IOU F1 : 0.040685135272911704
Token F1 : 0.19611300644443272
AUPRC : nan

Faithfulness
Comprehensiveness : 0.16766256511591043
Sufficiency -0.08029409333660155
